In [ ]:
import sqlite3

In [ ]:
import sqlite3
from typing import List, Dict, Any

# Open a connection to the database
def open_db(filename: str) -> sqlite3.Connection:
    connection = sqlite3.connect(filename)
    return connection


# Function to get messages by channel ID including user info
def get_messages_by_channel_id(connection: sqlite3.Connection, channel_id: str) -> List[Dict[str, Any]]:
    cursor = connection.cursor()
    cursor.execute('''
        SELECT m.id, m.channel_id, m.created_timestamp, m.content, m.clean_content, m.pinned, m.tts, m.nonce,
               u.id as user_id, u.username, u.discriminator, u.avatar
        FROM dms m
        JOIN users u ON m.author_id = u.id
        WHERE m.channel_id = ?
        ORDER BY m.created_timestamp DESC;
    ''', (channel_id,))
    return cursor.fetchall()

# Function to delete the 'dms' table
def delete_dms_table(connection: sqlite3.Connection):
    cursor = connection.cursor()
    cursor.execute('DROP TABLE IF EXISTS dms;')
    connection.commit()
    cursor.close()
    print('Table dms deleted successfully.')

# Function to delete the 'attachments' table
def delete_attachments_table(connection: sqlite3.Connection):
    cursor = connection.cursor()
    cursor.execute('DROP TABLE IF EXISTS attachments;')
    connection.commit()
    cursor.close()
    print('Table attachments deleted successfully.')


# Example usage
db_path = 'messages.db'
conn = open_db(db_path)




In [ ]:
messages = get_messages_by_channel_id(conn, 1048378429231337493)
for message in messages:
    print(message)

In [ ]:
# To delete the 'dms' table, you would call:
delete_dms_table(conn)
delete_attachments_table(conn)


In [ ]:
def get_messages_with_attachments_flag(connection: sqlite3.Connection, channel_id: str) -> List[Dict[str, Any]]:
    cursor = connection.cursor()
    cursor.execute('''
        SELECT id, channel_id, created_timestamp, content, clean_content,
               author_id, user_name, global_name, pinned, tts, nonce
        FROM dms
        WHERE channel_id = ? AND has_attachments = 1
        ORDER BY created_timestamp DESC;
    ''', (channel_id,))
    results = cursor.fetchall()
    cursor.close()
    return [{
        'id': row[0],
        'channel_id': row[1],
        'created_timestamp': row[2],
        'content': row[3],
        'clean_content': row[4],
        'author_id': row[5],
        'user_name': row[6],
        'global_name': row[7],
        'pinned': row[8],
        'tts': row[9],
        'nonce': row[10]
    } for row in results]

# Example usage
channel_id = 'example_channel_id'
conn = open_db('messages.db')
messages_with_attachments_flag = get_messages_with_attachments_flag(conn, channel_id)
for message in messages_with_attachments_flag:
    print(message)

conn.close()


In [ ]:
import sqlite3
from typing import List, Dict, Any

# Function to open a database connection
def open_db(filename: str) -> sqlite3.Connection:
    return sqlite3.connect(filename)

# Function to print URLs of attachments for messages that have them
def print_message_attachments(db_filename: str):
    # Open database connection
    conn = open_db(db_filename)
    
    # Prepare SQL query to find messages with attachments and their corresponding URLs
    query = '''
    SELECT dms.clean_content, attachments.url
    FROM dms
    JOIN attachments ON dms.id = attachments.message_id
    WHERE dms.has_attachments = 1;
    '''
    
    # Execute the query
    cursor = conn.cursor()
    try:
        cursor.execute(query)
        attachments_info = cursor.fetchall()
        
        # Check if there are any results
        if attachments_info:
            for content, url in attachments_info:
                print(f"Message: {content}\nAttachment URL: {url}\n")
        else:
            print("No messages with attachments found.")
    except sqlite3.Error as e:
        print(f"An error occurred: {e}")
    finally:
        cursor.close()
        conn.close()

# Example usage
db_path = 'messages.db'  # Update this to your actual database path
print_message_attachments(db_path)


In [ ]:

function cleanContent(message, name) {
    const botName = name;
    if (message.startsWith(`@${botName}`)) {
        return message.replace(new RegExp(`@${botName}`, "gi"), "").trim();
      } else {
        return message.replace("@", "").trim();
      }
}


In [ ]:
cleanContent("how are you?", "botName") // "Hello , how are you?"

In [ ]:
import sqlite3

def open_db(filename: str) -> sqlite3.Connection:
    """Open a connection to the database."""
    return sqlite3.connect(filename)

def get_last_x_messages(conn: sqlite3.Connection, channel_id: str, x: int):
    """Retrieve the last X messages for a specified channel ordered from oldest to newest,
    using user_name if global_name is None or Null. Fetches the bottom X messages."""
    cursor = conn.cursor()
    
    query = '''
        SELECT name, clean_content FROM (
            SELECT COALESCE(global_name, user_name) AS name, clean_content, created_timestamp
            FROM dms
            WHERE channel_id = ?
            ORDER BY created_timestamp DESC
            LIMIT ?
        ) sub ORDER BY created_timestamp ASC
    '''
    
    cursor.execute(query, (channel_id, x))
    messages = cursor.fetchall()
    
    cursor.close()
    return messages


# Example usage
db_path = 'messages.db'
conn = open_db(db_path)
channel_id = '1080579498749214861'  # Replace with your actual channel ID
x = 10  # Number of messages you want to retrieve

messages = get_last_x_messages(conn, channel_id, x)
for message in messages:
    print(message)

conn.close()


# adding column to db

In [1]:
import sqlite3
db_path = 'messages.db'

# Connect to the database
conn = sqlite3.connect(db_path)

# Create a cursor object
cur = conn.cursor()

# SQL statement to add a new column
alter_table_sql = '''
ALTER TABLE dms
ADD COLUMN `image_caption` TEXT;
'''

try:
    # Execute the SQL statement
    cur.execute(alter_table_sql)

    # Commit the changes
    conn.commit()
    print("Column 'image_caption' added successfully.")
except sqlite3.Error as e:
    print(f"An error occurred: {e}")
finally:
    # Close the connection
    conn.close()

Column 'image_caption' added successfully.


In [ ]:
import requests
import base64
from PIL import Image
from io import BytesIO

# Replace with the actual path to your image file
image_path = '1710009654635280.jpg'

# Read the image, convert it to RGB, and resize it if necessary
with open(image_path, 'rb') as image_file:
    image = Image.open(image_file)
    image = image.convert("RGB")
    buffered = BytesIO()
    image.save(buffered, format="JPEG")

# Encode the image to base64
image_base64 = base64.b64encode(buffered.getvalue()).decode('utf-8')

# Prepare the JSON payload
payload = {
    "image": image_base64
}

# Specify the URL of the API
url = "http://localhost:5100/api/caption"

# Send the POST request to the server
response = requests.post(url, json=payload)

# Check the response from the server
if response.status_code == 200:
    # The request was successful, print the caption
    print("Caption:", response.json()["caption"])
    # If you want to also retrieve the thumbnail, you can decode it like this:
    # thumbnail_base64 = response.json()["thumbnail"]
    # thumbnail = Image.open(BytesIO(base64.b64decode(thumbnail_base64)))
    # thumbnail.show()
else:
    # There was an error
    print("Error:", response.status_code, response.text)



In [ ]:
!pip install requests pillow